In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet,self).__init__()
      #we using gray scale image in this case so input channel is 1
      #out channels secound parameter is number pf feature maps
      #last one is kernel
    self.conv1=nn.Conv2d(1, 6, 3)
    self.conv2=nn.Conv2d(6,16,3)
      #add fully conencted layer
    self.fc1=nn.Linear(16*6*6,120) # 6 *6 for image size 120 is output
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)#softmax
  def forward(self,x):
    #max pool over a 2,2 window
    x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
    x=F.max_pool2d(F.relu(self.conv2(x)),(2))
    #if size is square you can only  specify a specifc number
    x=x.view(-1,self.num_flat_features(x)) #infer from data and use the other vlaue to resize in this case a flat one
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
  def num_flat_features(self,x):
    size=x.size()[1:] 
    # all dimenions except the batch dimension
    num_features = 1
    for s in size:
            num_features *= s
    return num_features

In [3]:
net=NeuralNet()
print(net)

NeuralNet(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())#learnable parameters are stored in .parameters
print(len(params))
print(params[0].size()) #weights

10
torch.Size([6, 1, 3, 3])


In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0333, -0.0050,  0.0733, -0.0045,  0.0793,  0.0538, -0.0758, -0.0772,
         -0.1246, -0.0929]], grad_fn=<AddmmBackward>)


In [7]:
#loss function
output=net(input)
target=torch.rand(10)
target=target.view(1,-1) #1 dimensional 1,10 ,-1 to infer the value
criterion=nn.MSELoss()
loss=criterion(output,target)

In [10]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0051,  0.0052, -0.0057, -0.0063, -0.0047,  0.0004])


In [11]:
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [12]:
import torch.optim as optim

optimizer=optim.SGD(net.parameters(),lr=0.01)
output=net(input)
loss=criterion(output,target)
loss.backward()
optimizer.step()